In [1]:
import pandas as pd 
import matplotlib.pyplot as plt 
import numpy as np 
import mediapipe as mp 
import cv2
import time

In [4]:
cap = cv2.VideoCapture(r"C:\Users\pc\OneDrive\Desktop\mediapip\Eye tracking 2\Right Eye 2025-01-28 09-31-59.mp4")

while True: 
    success,frame=cap.read()
    if success: 
        rows,cols, _ =frame.shape
        gray_frame=cv2.cvtColor(frame,cv2.COLOR_BGR2GRAY)
        gray_frame=cv2.GaussianBlur(gray_frame,(7,7),0)

        _, threshold =cv2.threshold(gray_frame,3,255,cv2.THRESH_BINARY_INV)
        contours, _ =cv2.findContours(threshold,cv2.RETR_TREE,cv2.CHAIN_APPROX_SIMPLE)
        contour=sorted(contours,key=lambda x:cv2.contourArea(x),reverse=True)

        for cnt in contour: 
            (x,y,w,h)=cv2.boundingRect(cnt)

            cv2.rectangle(frame,(x,y),(x+w,y+h),(255,0,0),2)
            cv2.line(frame,(x+int(w/2),0),(x+int(w/2),rows),(0,255,0),2)
            print(int(w/2))
            cv2.line(frame,(0,y+int(h/2)),(cols,y+int(h/2)),(0,255,0),2)
            break
        cv2.imshow("gray_frame",gray_frame)
        cv2.imshow("frame",frame)
        key=cv2.waitKey(30)
        if key==27:
            break
cv2.destroyAllWindows()

85
85
91
92
91
91
92
91
91
89
89
89
92
92
90
90
89
90
102
102
104
104
105
103
105
105
105
106
104
104
104
103
102
102
99
99
98
97
93
94
93
93
92
93
95
95
95
102
102
105
105
106
106
111
107
107
107
107
108
102
110
110
110
67
68
68
68
75
74
78
72
78
77
75
73
73
75
74
69
69
69
74
109
91
91
91
89
91
94
90
89
88
88
86
90
90
89
87
86
86
86
85
88
72
86
86
88
87
83
72
72
86
91
114
112
112
109
108
108
114
114
111
112
112
110
110
81
73
73
70
70
70
71
72
71
71
71
75
88
125
125
124
121
121
116
116
116
113
108
109
109
108
106
104
104
108
108
109
107
110
110
108
108
106
105
102
102
101
100
102
102
102
109
107
108
108
108
108
108
112
106
106
106
106
106


In [ ]:
import cv2
import mediapipe as mp
import time

# Initialize variables
direction = "Unknown"

mp_face_mesh = mp.solutions.face_mesh
face_mesh = mp_face_mesh.FaceMesh()

cap = cv2.VideoCapture(0)

# Indices for specific landmarks
left_eye = [285, 276, 450]
right_eye = [46, 55, 230]

while True:
    success, frame = cap.read()
    if not success:
        print("Failed to capture frame.")
        break

    height, width, _ = frame.shape
    frame_rgb = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
    results = face_mesh.process(frame_rgb)

    left_eye_frame, right_eye_frame = None, None

    if results.multi_face_landmarks:
        for face_landmarks in results.multi_face_landmarks:
            try:
                # Get coordinates for left eye
                x_left_eye_left = int(face_landmarks.landmark[left_eye[0]].x * width)
                x_left_eye_right = int(face_landmarks.landmark[left_eye[1]].x * width)
                y_left_eye_up = int(face_landmarks.landmark[left_eye[0]].y * height)
                y_left_eye_down = int(face_landmarks.landmark[left_eye[2]].y * height)

                # Extract and resize the left eye region
                left_eye_frame = frame[y_left_eye_up:y_left_eye_down, x_left_eye_left:x_left_eye_right]
                left_eye_frame = cv2.resize(left_eye_frame, (600, 400))

                # Process the left eye frame
                rows, cols, _ = left_eye_frame.shape
                gray_frame = cv2.cvtColor(left_eye_frame, cv2.COLOR_BGR2GRAY)
                gray_frame = cv2.GaussianBlur(gray_frame, (7, 7), 0)

                _, threshold = cv2.threshold(gray_frame, 3, 255, cv2.THRESH_BINARY_INV)
                contours, _ = cv2.findContours(threshold, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
                contours = sorted(contours, key=lambda x: cv2.contourArea(x), reverse=True)

                if contours:
                    for cnt in contours:
                        (x, y, w, h) = cv2.boundingRect(cnt)
                        cv2.rectangle(left_eye_frame, (x, y), (x + w, y + h), (255, 0, 0), 2)
                        cv2.line(left_eye_frame, (x + int(w / 2), 0), (x + int(w / 2), rows), (0, 255, 0), 2)
                        cv2.line(left_eye_frame, (0, y + int(h / 2)), (cols, y + int(h / 2)), (0, 255, 0), 2)

                        # Determine direction based on horizontal (x) and vertical (y) position
                        center_x = x + w // 2
                        center_y = y + h // 2

                        if center_x < cols // 3:
                            if center_y < rows // 3:
                                direction = "Up-Right"
                            elif center_y > 2 * rows // 3:
                                direction = "Down-Right"
                            else:
                                direction = "Right"
                        elif center_x > 2 * cols // 3:
                            if center_y < rows // 3:
                                direction = "Up-Left"
                            elif center_y > 2 * rows // 3:
                                direction = "Down-Left"
                            else:
                                direction = "Left"
                        else:
                            if center_y < rows // 3:
                                direction = "Up"
                            elif center_y > 2 * rows // 3:
                                direction = "Down"
                            else:
                                direction = "Forward"
                        break
            except IndexError:
                print("Error accessing landmarks. Skipping this frame.")
                continue

    # Display the main frame with direction text
    cv2.putText(frame, direction, (50, 100), cv2.FONT_HERSHEY_SIMPLEX, 1, (255, 0, 0), 2)
    cv2.imshow("Window", frame)

    # Display the left eye frame if valid
    if left_eye_frame is not None:
        cv2.imshow("Left Eye", left_eye_frame)

    time.sleep(0.001)

    # Exit on pressing the ESC key
    if cv2.waitKey(1) == 27:
        break

# Release resources
cap.release()
cv2.destroyAllWindows()
